In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

import warnings
warnings.simplefilter('ignore')

# general purpose packages
import pandas as pd
import numpy as np
import os
import json
import time
import re
import csv
import subprocess
import sys

import scipy.stats as stats
import statsmodels.stats as smstats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import umap
import rpy2

from multiprocessing import Process, Manager, Pool
import multiprocessing
from functools import partial

from collections import Counter

import seaborn as sns; sns.set()

import matplotlib
matplotlib.style.use('seaborn')
matplotlib.use('Agg')
import matplotlib.pyplot as plt
matplotlib.rcParams['backend'] = "Qt5Agg"
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

from IPython.display import display, Image

from adjustText import adjust_text
import builtins
%matplotlib inline

# for normalization
from sklearn.linear_model import QuantileRegressor

# for working with yaml files
import ruamel.yaml

# for working with genomic intervals
import pyranges as pr

import itertools

In [2]:
def get_pvalue_star(pval, thr=0.05):
    if thr == 0.05:
        if pval < 0.001:
            return "***"
        elif pval < 0.01:
            return "**"
        elif pval < 0.05:
            return "*"
        else:
            return "ns"
    elif thr == 0.1:
        if pval < 0.001:
            return "***"
        elif pval < 0.01:
            return "**"
        elif pval < 0.1:
            return "*"
        else:
            return "ns"

In [3]:
# paths to subdirectories
subdirs = {}

subdirs['main_project_dir'] = '/scicore/home/zavolan/GROUP/StefanieCLIP/'
subdirs['wf_dir'] = '/scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/'

subdirs['zarp_dir'] = '/scicore/home/zavolan/mirono0000/libs/zarp/'
subdirs['htsinfer_dir'] = '/scicore/home/zavolan/mirono0000/libs/htsinfer/'
subdirs['zarp_config_dir'] = subdirs['zarp_dir']+'config/'

subdirs['mouse_annotation_dir'] = '/scicore/home/zavolan/GROUP/Genomes/mus_musculus/'
subdirs['human_annotation_dir'] = '/scicore/home/zavolan/GROUP/Genomes/homo_sapiens/'

# shared project folder 
subdirs['shared_project_dir'] = subdirs['main_project_dir']+'aleksei/'
subdirs['temp_dir'] = subdirs['shared_project_dir']+'temp_dir/'
subdirs['slurm_dir'] = subdirs['temp_dir']+'slurm/'
subdirs['scripts_dir'] = subdirs['shared_project_dir']+'scripts/'
subdirs['figures_dir'] = subdirs['shared_project_dir']+'figures/'

subdirs['fastq_dir'] = subdirs['shared_project_dir']+'input_fastq/'
subdirs['metadata_dir'] = subdirs['shared_project_dir']+'metadata/'
subdirs['wf_output_dir'] = subdirs['shared_project_dir']+'output/'

# paths to files
file_paths = {}
### genome annotation files
file_paths['mouse_genome_file'] = subdirs['mouse_annotation_dir']+'GRCm39.primary_assembly.genome.fa'
file_paths['mouse_annotation_file'] = subdirs['mouse_annotation_dir']+'gencode.vM32.annotation.gtf'
file_paths['mouse_RNAcentral_annotation_file'] = subdirs['mouse_annotation_dir']+'mus_musculus.GRCm39.gff3.gz'

file_paths['mouse_prot_coding_gtf'] = subdirs['mouse_annotation_dir']+'coding.gencode.vM32.annotation.gtf'
file_paths['mouse_collapsed_prot_coding_gtf'] = subdirs['mouse_annotation_dir']+'collapsed.coding.gencode.vM32.annotation.gtf'

file_paths['mouse_enriched_annotation_file'] = subdirs['mouse_annotation_dir']+'enriched.gencode.vM32.annotation.gtf' # added RNA species from RNA central
file_paths['mouse_collapsed_enriched_annotation_file'] = subdirs['mouse_annotation_dir']+'collapsed.enriched.gencode.vM32.annotation.gtf'

file_paths['human_genome_file'] = subdirs['human_annotation_dir']+'GRCh38.primary_assembly.genome.fa'
file_paths['human_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/gencode.v42.annotation.gtf'
file_paths['human_RNAcentral_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/homo_sapiens.GRCh38.gff3.gz'

file_paths['human_enriched_annotation_file'] = subdirs['human_annotation_dir']+'hg38_v42/enriched.gencode.v42.annotation.gtf'

file_paths['htsinfer_transcripts_file'] = '/scicore/home/zavolan/GROUP/Genomes/htsinfer_deduplicated_transcripts.fasta'

### control data
file_paths['control_metadata_file_1'] = subdirs['metadata_dir']+'SRA_CONTROL_DATA_1.tsv'
file_paths['control_metadata_file_2'] = subdirs['metadata_dir']+'SRA_CONTROL_DATA_2.tsv'

os.system('mkdir -p '+' '.join(list(subdirs.values()))) # create all subdirs

0

# move bam files to dedicated folders

In [6]:
os.system("""find """+subdirs['wf_output_dir']+'samples/'+""" -name '*.0_um.sorted.bam' > """+subdirs['temp_dir']+"""low_dupl_files.tsv""")

0

In [8]:
os.system("""find """+subdirs['wf_output_dir']+'samples/'+""" -name '*.dedup.sorted.indexed.low_duplicate.bam' > """+subdirs['temp_dir']+"""low_dupl_files_1.tsv""")

0

In [24]:
low_dupl_files = pd.read_csv(subdirs['temp_dir']+"""low_dupl_files.tsv""",delimiter="\t",
                                   index_col=None,header=None)
low_dupl_files_1 = pd.read_csv(subdirs['temp_dir']+"""low_dupl_files_1.tsv""",delimiter="\t",
                                   index_col=None,header=None)
low_dupl_files['prior'] = 1
low_dupl_files_1['prior'] = 2
low_dupl_files = pd.concat([low_dupl_files,low_dupl_files_1]).reset_index(drop=True)
low_dupl_files['sample'] = low_dupl_files.apply(lambda x:x[0].split('/samples/')[1].split('/')[0],1)
low_dupl_files = low_dupl_files.sort_values(['sample','prior']).drop_duplicates('sample').reset_index(drop=True)
low_dupl_files[[0]].to_csv(subdirs['temp_dir']+"""low_dupl_files_to_retain.tsv""", sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

In [34]:
low_dupl_files[1] = low_dupl_files[0]+'.bai'

In [35]:
low_dupl_files[[1]].to_csv(subdirs['temp_dir']+"""low_dupl_bai_files_to_retain.tsv""", sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

In [11]:
low_dupl_files.iloc[0][0]

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/N4BP2C_replicate1/map_genome_all/read_categories/0_um/N4BP2C_replicate1.0_um.sorted.bam'

In [12]:
low_dupl_files_1.iloc[0][0]

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/samples/N4BP2C_replicate1/map_genome_all/N4BP2C_replicate1.dedup.sorted.indexed.low_duplicate.bam'

In [7]:
subdirs['temp_dir']+"""low_dupl_files.tsv"""

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/low_dupl_files.tsv'

In [32]:
"mkdir -p ./low_dupl_bam_files && cat "+subdirs['temp_dir']+"""low_dupl_files_to_retain.tsv"""+" | xargs -I % cp % "+'./low_dupl_bam_files'

'mkdir -p ./low_dupl_bam_files && cat /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/low_dupl_files_to_retain.tsv | xargs -I % cp % ./low_dupl_bam_files'

In [38]:
"mkdir -p ./low_dupl_bam_files && cat "+subdirs['temp_dir']+"""low_dupl_bai_files_to_retain.tsv"""+" | xargs -I % cp % "+'./low_dupl_bam_files'

'mkdir -p ./low_dupl_bam_files && cat /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/low_dupl_bai_files_to_retain.tsv | xargs -I % cp % ./low_dupl_bam_files'

In [40]:
"""tar -czvf low_duplication_bam_files.tar.gz ./low_dupl_bam_files"""

'tar -czvf low_duplication_bam_files.tar.gz ./low_dupl_bam_files'

# Make enriched gtf file for mouse and human

In [4]:
# organisms = ['mouse','human']
organisms = ['mouse']

for organism in organisms:
    command = 'samtools faidx '+file_paths[organism+'_genome_file']
    out = subprocess.check_output(command, shell=True)

    gtf_df = pd.read_csv(file_paths[organism+'_annotation_file'],delimiter="\t",index_col=None,header=None,skiprows=5)

    rna_central = pd.read_csv(file_paths[organism+'_RNAcentral_annotation_file'],delimiter="\t",index_col=None,header=None,skiprows=1,compression='gzip')
    rna_central[2] = rna_central[2].str.replace('noncoding_exon','exon')
    rna_central['gene_biotype'] = rna_central[8].str.split(';type=|;',expand=True)[1]
    rna_central['gene_source'] = 'RNA_central'
    rna_central['gene_id'] = rna_central[8].str.split('ID=|;|:',expand=True)[4]

    rna_central_exons = rna_central.loc[rna_central[2]=='exon'].copy().reset_index(drop=True)
    rna_central_exons['exon_id'] = rna_central_exons['gene_id']+'.transcript'+'.'+rna_central_exons[8].str.split('ID=|;|:',expand=True)[5]
    rna_central_exons['exon_number'] = rna_central_exons[8].str.split('ID=|;|:',expand=True)[5].str.split('exon',expand=True).iloc[:, -1]
    rna_central_exons[8] = 'gene_id "'+rna_central_exons['gene_id']+'"; transcript_id "'+rna_central_exons['gene_id']+'.transcript'+'"; exon_number "'+rna_central_exons['exon_number']+'"; gene_source "'+rna_central_exons['gene_source']+'"; gene_type "'+rna_central_exons['gene_biotype']+'"; transcript_source "'+rna_central_exons['gene_source']+'"; transcript_type "'+rna_central_exons['gene_biotype']+'"; exon_id "'+rna_central_exons['exon_id']+'"; tag "'+rna_central['gene_source']+'";'
    rna_central_exons['order']=3

    rna_central_exons['transcript_id'] = rna_central_exons['gene_id']+'.transcript'
    rna_central_exons['exon_coords'] = rna_central_exons[3].astype('str')+'_'+rna_central_exons[4].astype('str')+','
    rna_central_gr_transcripts = rna_central_exons.groupby([0,6,'transcript_id']).agg({'exon_coords':sum}).reset_index()
    rna_central_gr_transcripts['transcript_alt_id'] = rna_central_gr_transcripts[0].astype('str')+'_'+rna_central_gr_transcripts[6]+'_'+rna_central_gr_transcripts['exon_coords']

    gtf_df_exons = gtf_df.loc[gtf_df[2]=='exon'].reset_index(drop=True)
    gtf_df_exons['transcript_id'] = gtf_df_exons[8].str.split('transcript_id "',expand=True)[1].str.split('"',expand=True)[0]
    gtf_df_exons['exon_coords'] = gtf_df_exons[3].astype('str')+'_'+gtf_df_exons[4].astype('str')+','
    gtf_df_gr_transcripts = gtf_df_exons.groupby([0,6,'transcript_id']).agg({'exon_coords':sum}).reset_index()
    gtf_df_gr_transcripts['transcript_alt_id'] = gtf_df_gr_transcripts[0].astype('str')+'_'+gtf_df_gr_transcripts[6]+'_'+gtf_df_gr_transcripts['exon_coords']
    ensembl_transcripts = list(gtf_df_gr_transcripts['transcript_alt_id'].unique())

    preserve_transcripts_list = list(rna_central_gr_transcripts.loc[~rna_central_gr_transcripts['transcript_alt_id'].isin(ensembl_transcripts)]['transcript_id'].unique()) # when a transcript is present in ensemble and RNA central, prioritize ensemble
    rna_central_exons = rna_central_exons.loc[rna_central_exons['transcript_id'].isin(preserve_transcripts_list)].reset_index(drop=True)
    rna_central['transcript_id'] = rna_central['gene_id']+'.transcript'
    rna_central = rna_central.loc[rna_central['transcript_id'].isin(preserve_transcripts_list)].reset_index(drop=True)

    rna_central_exons = rna_central_exons[list(range(0,9))+['gene_id','order']]

    rna_central_transcripts = rna_central.loc[rna_central[2]=='transcript'].copy().reset_index(drop=True)
    rna_central_transcripts[8] = 'gene_id "'+rna_central_transcripts['gene_id']+'"; transcript_id "'+rna_central_transcripts['gene_id']+'.transcript'+'"; gene_source "'+rna_central_transcripts['gene_source']+'"; gene_type "'+rna_central_transcripts['gene_biotype']+'"; transcript_source "'+rna_central_transcripts['gene_source']+'"; transcript_type "'+rna_central_transcripts['gene_biotype']+'"; tag "'+rna_central_transcripts['gene_source']+'";'
    rna_central_transcripts['order']=2
    rna_central_transcripts = rna_central_transcripts[list(range(0,9))+['gene_id','order']]

    rna_central_genes = rna_central.loc[rna_central[2]=='transcript'].copy().reset_index(drop=True)
    rna_central_genes[2] = 'gene'
    rna_central_genes[8] = 'gene_id "'+rna_central_genes['gene_id']+'"; gene_source "'+rna_central_genes['gene_source']+'"; gene_type "'+rna_central_genes['gene_biotype']+'";'
    rna_central_genes['order']=1
    rna_central_genes = rna_central_genes[list(range(0,9))+['gene_id','order']]

    rna_central_gtf = pd.concat([rna_central_genes,rna_central_transcripts,rna_central_exons]).sort_values(['gene_id','order']).reset_index(drop=True).drop(['gene_id','order'],1)
    rna_central_gtf[0] = 'chr'+rna_central_gtf[0].astype('str') # so that the naming is consistent
    # save standard annotation enriched with RNA central
    enriched_gtf = pd.concat([gtf_df,rna_central_gtf]).reset_index(drop=True)
    genome_fai = pd.read_csv(file_paths[organism+'_genome_file']+'.fai',delimiter="\t",index_col=None,header=None)
    enriched_gtf = pd.merge(genome_fai[[0]],enriched_gtf,how='inner',on=0)

    # remove non-canonical chromosomes, they are not of interest
    enriched_gtf = enriched_gtf.loc[enriched_gtf[0].str.startswith('chr')].reset_index(drop=True)
    
    # check that inferred introns are consistent (start<end), otherwise remove such transcripts
    exons = enriched_gtf.loc[enriched_gtf[2]=='exon'].reset_index(drop=True)
    exons['transcript_id'] = exons[8].str.split('transcript_id "',expand=True)[1].str.split('";',expand=True)[0]
    exons = exons[[0,3,4,'transcript_id',6]]
    exons = exons.sort_values([0,6,'transcript_id',3,4],ascending=True).reset_index(drop=True)
    exons = exons.values
    introns = []
    intron_start, intron_end, prev_transcript = None, None, ''
    
    for exon in exons:
        if (intron_start is None) or (prev_transcript!=exon[3]):
            intron_start = exon[2]
            prev_transcript = exon[3]
        else:
            intron_end = exon[1]
            introns.append([exon[0],intron_start,intron_end,exon[3],0,exon[4]])
            intron_start = exon[2]
            prev_transcript = exon[3]
    introns = pd.DataFrame(introns)
    introns[2] = introns[2]-1
    introns = introns.sort_values([0,1,2],ascending=True).reset_index(drop=True)
    
    bad_transcripts = list(introns.loc[introns[1]>introns[2]][3].unique())
    
    transcripts = enriched_gtf.loc[enriched_gtf[2]=='transcript'].reset_index(drop=True)
    transcripts['transcript_id'] = transcripts[8].str.split('transcript_id "',expand=True)[1].str.split('";',expand=True)[0]
    transcripts['gene_id'] = transcripts[8].str.split('gene_id "',expand=True)[1].str.split('";',expand=True)[0]
    transcripts['bad'] = (transcripts['transcript_id'].isin(bad_transcripts)).astype('int')
    transcripts['t']=1
    gr = transcripts.groupby(['gene_id']).agg({'t':sum,'bad':sum}).reset_index()
    
    bad_genes = list(gr.loc[(gr['t']==gr['bad'])&(gr['bad']>0)]['gene_id'].unique())
    
    if len(bad_genes)>0:
        enriched_gtf = enriched_gtf.loc[~enriched_gtf[8].str.contains('|'.join(bad_genes))].reset_index(drop=True)
    if len(bad_transcripts):
        enriched_gtf = enriched_gtf.loc[~enriched_gtf[8].str.contains('|'.join(bad_transcripts))].reset_index(drop=True)
    
    enriched_gtf.to_csv(file_paths[organism+'_enriched_annotation_file'], sep=str('\t'),header=False,index=None,quoting=csv.QUOTE_NONE)

# Download control data<a name="control_data_download"></a>

In [18]:
samples_to_download_1 = pd.read_csv(subdirs['metadata_dir']+'SRA_CONTROL_DATA_1.tsv',delimiter="\t",
                                   index_col=None,header=0)
samples_to_download_2 = pd.read_csv(subdirs['metadata_dir']+'SRA_CONTROL_DATA_2.tsv',delimiter="\t",
                                   index_col=None,header=0)
samples_to_download_3 = pd.read_csv(subdirs['metadata_dir']+'SRA_CONTROL_DATA_3.tsv',delimiter="\t",
                                   index_col=None,header=0)
samples_to_download = pd.concat([samples_to_download_1,samples_to_download_2,samples_to_download_3]).reset_index(drop=True)

In [21]:
samples_to_download['expected_path'] = subdirs['fastq_dir']+samples_to_download['sample']+'/'+samples_to_download['sample']

In [27]:
big_command = ''
i=0
for index,row in samples_to_download.iterrows():
    SRA_id = row['sample']
    if not (os.path.isfile(row['expected_path']+'.fastq.gz') or os.path.isfile(row['expected_path']+'_1.fastq.gz')):
        command = 'prefetch -v -O '+subdirs['fastq_dir']+SRA_id+' '+SRA_id+\
        ' && fasterq-dump -f -v -e '+str(4)+' -O '+subdirs['fastq_dir']+SRA_id+' '+subdirs['fastq_dir']+SRA_id+'/'+SRA_id
        if i>0:
            big_command = big_command+' && '+command
        else:
            big_command = command
        i=i+1
big_command

'prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735700 SRR18735700 && fasterq-dump -f -v -e 4 -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735700 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735700/SRR18735700 && prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702 SRR18735702 && fasterq-dump -f -v -e 4 -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702/SRR18735702 && prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704 SRR18735704 && fasterq-dump -f -v -e 4 -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704/SRR18735704 && prefetch -v -O /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735706 SRR18735706 && fasterq-dump -f -v -e 4

In [35]:
# gzip fastq files
os.system("""find """+subdirs['fastq_dir']+""" -name '*.fastq' > """+subdirs['temp_dir']+"""fastq_file_paths.tsv""")
fastq_file_paths = pd.read_csv(subdirs['temp_dir']+"""fastq_file_paths.tsv""",delimiter="\t",
                                   index_col=None,header=None)
big_command = ''
i=0
for file_path in fastq_file_paths[0]:
    command = 'pigz -f --fast -p '+str(12)+' '+file_path
    if i>0:
        big_command = big_command+' && '+command
    else:
        big_command = command    
    i=i+1
big_command

'pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702/SRR18735702_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735702/SRR18735702_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704/SRR18735704_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735704/SRR18735704_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735698/SRR18735698_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735698/SRR18735698_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735706/SRR18735706_1.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/input_fastq/SRR18735706/SRR18735706_2.fastq && pigz -f --fast -p 12 /scicore/home/zavolan/GROUP/StefanieCLIP/a

In [36]:
# delete unnecessary .sra files
os.system("""find """+subdirs['fastq_dir']+""" -name '*.sra' > """+subdirs['temp_dir']+"""sra_file_paths.tsv""")
sra_file_paths = pd.read_csv(subdirs['temp_dir']+"""sra_file_paths.tsv""",delimiter="\t",
                                   index_col=None,header=None)
sra_file_paths.columns = ['path']
command = 'rm '+' '.join(sra_file_paths['path'])
# out = subprocess.check_output(command, shell=True)

In [ ]:
# make downloaded fastq.gz files read-only

In [ ]:
# chmod 554 -R ./input_fastq

# Make start_samples file

## for bCLIP and ChipSeq samples

In [39]:
# description file for bCLIP experimental samples
experimental_samples = pd.read_csv(subdirs['metadata_dir']+'experimental_bCLIP_samples.tsv',delimiter="\t",
                                   index_col=None,header=0)
exp_file_paths = experimental_samples[['lane_file','barcode_file']].drop_duplicates().reset_index(drop=True)

os.system("""find """+subdirs['main_project_dir']+'input/'+""" -name '*.fastq.gz' > """+subdirs['temp_dir']+"""exper_fastq_file_paths.tsv""")
fastq_file_paths = pd.read_csv(subdirs['temp_dir']+'exper_fastq_file_paths.tsv',delimiter="\t",
                                   index_col=None,header=None)
fastq_file_paths.columns = ['lane_file_path']
fastq_file_paths['lane_file'] = fastq_file_paths['lane_file_path'].str.split('/',expand=True).iloc[:, -1]
exp_file_paths = pd.merge(fastq_file_paths,exp_file_paths,how='inner',on=['lane_file'])

os.system("""find """+subdirs['main_project_dir']+'input/'+""" -name '*.fasta' > """+subdirs['temp_dir']+"""barcode_fasta_files.tsv""")
barcode_fasta_files = pd.read_csv(subdirs['temp_dir']+'barcode_fasta_files.tsv',delimiter="\t",
                                   index_col=None,header=None)
barcode_fasta_files.columns = ['barcode_file_path']
barcode_fasta_files['barcode_file'] = barcode_fasta_files['barcode_file_path'].str.split('/',expand=True).iloc[:, -1]
exp_file_paths = pd.merge(barcode_fasta_files,exp_file_paths,how='inner',on=['barcode_file'])

exp_file_paths['wfstart_lane_file_path'] = subdirs['fastq_dir']+exp_file_paths['lane_file'].str.replace('.fastq.gz','')+'/'+exp_file_paths['lane_file']
exp_file_paths['wfstart_barcode_file_path'] = subdirs['fastq_dir']+exp_file_paths['lane_file'].str.replace('.fastq.gz','')+'/'+exp_file_paths['barcode_file']

# for index, row in exp_file_paths.iterrows():
#     sample_name = row['lane_file'].replace('.fastq.gz','')
#     command = 'mkdir -p '+subdirs['fastq_dir']+sample_name+'/'
#     out = subprocess.check_output(command, shell=True)
#     command = 'rm -f '+row['wfstart_lane_file_path']+' && ln -f -s '+row['lane_file_path']+' '+row['wfstart_lane_file_path']
#     out = subprocess.check_output(command, shell=True)
#     command = 'rm -f '+row['wfstart_barcode_file_path']+' && ln -f -s '+row['barcode_file_path']+' '+row['wfstart_barcode_file_path']
#     out = subprocess.check_output(command, shell=True)

experimental_samples = pd.merge(experimental_samples,
                                exp_file_paths[['lane_file','barcode_file','wfstart_lane_file_path','wfstart_barcode_file_path']],
                                how='inner',on=['lane_file','barcode_file'])

experimental_samples = experimental_samples.drop(['lane_file','barcode_file'],1).rename(columns={'wfstart_lane_file_path':'lane_file','wfstart_barcode_file_path':'barcode_file'})

experimental_samples['genome_file'] = file_paths['mouse_genome_file']
experimental_samples['gtf_file'] = file_paths['mouse_annotation_file']
experimental_samples['lane_file_2'] = ''
experimental_samples = experimental_samples[['name','lane_name','lane_file','lane_file_2','kmer','barcode_file','exp_ctl','batch','condition_name','genome_file','gtf_file']]

experimental_samples['lane_name'] = 'f'+experimental_samples['barcode_file'].str.split('barcodes_',expand=True).iloc[:, -1].str.replace('.fasta','')
experimental_samples['organism'] = 'mouse'
experimental_samples['directionality'] = 's'
experimental_samples['library_layout'] = 'SINGLE'

# description file for public bCLIP samples
public_samples = pd.read_csv(subdirs['metadata_dir']+'public_bCLIP_and_ChipSeq_samples.tsv',delimiter="\t",
                                   index_col=None,header=0)
public_samples['genome_file'] = public_samples.apply(lambda x:file_paths['mouse_genome_file'] if x['batch'].endswith('_mESC') else file_paths['human_genome_file'],1)
public_samples['gtf_file'] = public_samples.apply(lambda x:file_paths['mouse_annotation_file'] if x['batch'].endswith('_mESC') else file_paths['human_annotation_file'],1)
public_samples['lane_name'] = 'f'+public_samples['name']
public_samples['organism'] = public_samples.apply(lambda x: 'mouse' if x['batch'].endswith('_mESC') else 'human',1)
public_samples['directionality'] = 's' # forward orientation of the first read
public_samples['library_layout'] = 'SINGLE'

# merge with public data
start_samples = pd.concat([public_samples,experimental_samples]).reset_index(drop=True)
start_samples['kmer'] = start_samples['kmer'].fillna('XXXXXXX')

# add list of chromosomes
organisms = list(start_samples['organism'].unique())
cl = []
for organism in organisms:
    enriched_gtf = pd.read_csv(file_paths[organism+'_enriched_annotation_file'],delimiter="\t",
                                       index_col=None,header=None,usecols = [0])
    chromosome_list =  list(enriched_gtf[0].unique())
    cl.append([organism,' '.join(chromosome_list)])
cl = pd.DataFrame(cl,columns=['organism','chromosomes'])
start_samples = pd.merge(start_samples,cl,how='left',on='organism')

start_samples.to_csv(subdirs['metadata_dir']+'start_bCLIP_and_ChipSeq_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

In [41]:
subdirs['metadata_dir']+'start_bCLIP_samples.tsv'

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/metadata/start_bCLIP_samples.tsv'

In [47]:
subdirs['wf_output_dir']

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/'

In [48]:
subdirs['slurm_dir']

'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/temp_dir/slurm/'

## for PRO-seq

In [40]:
# description file for public PRO-seq samples
public_samples = pd.read_csv(subdirs['metadata_dir']+'public_PROseq_samples.tsv',delimiter="\t",
                                   index_col=None,header=0)

public_samples['genome_file'] = public_samples.apply(lambda x:file_paths['mouse_genome_file'] if x['batch'].endswith('_mESC') else file_paths['human_genome_file'],1)
public_samples['gtf_file'] = public_samples.apply(lambda x:file_paths['mouse_annotation_file'] if x['batch'].endswith('_mESC') else file_paths['human_annotation_file'],1)
public_samples['lane_name'] = 'f'+public_samples['name']
public_samples['organism'] = public_samples.apply(lambda x: 'mouse' if x['batch'].endswith('_mESC') else 'human',1)
public_samples['directionality'] = 'S' # reverse orientation of the first read
public_samples['library_layout'] = 'SINGLE'

start_samples = public_samples.copy()
start_samples['kmer'] = start_samples['kmer'].fillna('XXXXXXX')

# add list of chromosomes
organisms = list(start_samples['organism'].unique())
cl = []
for organism in organisms:
    enriched_gtf = pd.read_csv(file_paths[organism+'_enriched_annotation_file'],delimiter="\t",
                                       index_col=None,header=None,usecols = [0])
    chromosome_list =  list(enriched_gtf[0].unique())
    cl.append([organism,' '.join(chromosome_list)])
cl = pd.DataFrame(cl,columns=['organism','chromosomes'])
start_samples = pd.merge(start_samples,cl,how='left',on='organism')
start_samples.to_csv(subdirs['metadata_dir']+'start_PROseq_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

## for RNA-seq

In [31]:
os.system("""find """+subdirs['main_project_dir']+'input/X204SC24014174-Z01-F001/01.RawData/'+""" -name '*.fq.gz' > """+subdirs['temp_dir']+"""exper_RNAseq_fastq_file_paths.tsv""")

0

In [39]:
exper_RNAseq_fastq_file_paths = pd.read_csv(subdirs['temp_dir']+"""exper_RNAseq_fastq_file_paths.tsv""",delimiter="\t",index_col=None,header=None)
exper_RNAseq_fastq_file_paths.columns = ['path']
exper_RNAseq_fastq_file_paths['name'] = exper_RNAseq_fastq_file_paths.apply(lambda x:x['path'].split('/')[-1][:-8],1)
exper_RNAseq_fastq_file_paths['lane_file'] = exper_RNAseq_fastq_file_paths.apply(lambda x:'/'.join(x['path'].split('/')[:-1])+'/'+x['name']+'_1.fq.gz',1)
exper_RNAseq_fastq_file_paths['lane_file_2'] = exper_RNAseq_fastq_file_paths.apply(lambda x:'/'.join(x['path'].split('/')[:-1])+'/'+x['name']+'_2.fq.gz',1)
experimental_samples = exper_RNAseq_fastq_file_paths.drop_duplicates('name').reset_index(drop=True)
experimental_samples['replicate'] = experimental_samples.apply(lambda x:x['name'].split('_')[0],1)

mouse_replicates = ['A12','A13','A14','A15','A18','A19']
human_replicates = list('A'+pd.Series(range(1,10)).astype('str'))

experimental_samples = experimental_samples.loc[experimental_samples['replicate'].isin(mouse_replicates+human_replicates)].reset_index(drop=True).drop('path',1)

experimental_samples_mouse = experimental_samples.loc[experimental_samples['replicate'].isin(mouse_replicates)].reset_index(drop=True)
experimental_samples_human = experimental_samples.loc[experimental_samples['replicate'].isin(human_replicates)].reset_index(drop=True)
experimental_samples_mouse['genome_file'] = file_paths['mouse_genome_file']
experimental_samples_human['genome_file'] = file_paths['human_genome_file']
experimental_samples_mouse['gtf_file'] = file_paths['mouse_annotation_file']
experimental_samples_human['gtf_file'] = file_paths['human_annotation_file']
experimental_samples_human['organism'] = 'human'
experimental_samples_mouse['organism'] = 'mouse'

# experimental_samples = pd.concat([experimental_samples_human,experimental_samples_mouse]).reset_index(drop=True)
experimental_samples = experimental_samples_human.copy()

experimental_samples = experimental_samples.drop('replicate',1)
experimental_samples['condition_name'] = 'NA' # define afterwards in analysis
experimental_samples['batch'] = 'new24_03'
experimental_samples['exp_ctl'] = 'NA'

# public_samples = pd.read_csv(subdirs['metadata_dir']+'public_RNAseq_samples.tsv',delimiter="\t",
#                                    index_col=None,header=0)
# public_samples['genome_file'] = file_paths['mouse_genome_file']
# public_samples['gtf_file'] = file_paths['mouse_annotation_file']
# public_samples['organism'] = 'mouse'

# start_samples = pd.concat([public_samples[list(experimental_samples.columns)],experimental_samples]).reset_index(drop=True)
start_samples = experimental_samples.copy()
start_samples['kmer'] = 'XXXXXXX'


# add list of chromosomes
organisms = list(start_samples['organism'].unique())
cl = []
for organism in organisms:
    enriched_gtf = pd.read_csv(file_paths[organism+'_enriched_annotation_file'],delimiter="\t",
                                       index_col=None,header=None,usecols = [0])
    chromosome_list =  list(enriched_gtf[0].unique())
    cl.append([organism,' '.join(chromosome_list)])
cl = pd.DataFrame(cl,columns=['organism','chromosomes'])
start_samples = pd.merge(start_samples,cl,how='left',on='organism')

start_samples.to_csv(subdirs['metadata_dir']+'start_RNAseq_samples.tsv', sep=str('\t'),header=True,index=None,quoting=csv.QUOTE_NONE)

# Run main WF

## bCLIP and ChipSeq analysis

In [17]:
command = """snakemake \
--snakefile """+subdirs['wf_dir']+"""Snakefile_bCLIP \
--configfile """+subdirs['wf_dir']+"""config_bCLIP.yaml \
--scheduler greedy \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" \
--cluster-config """+subdirs['wf_dir']+"""cluster_bCLIP.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/Snakefile_bCLIP --configfile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/config_bCLIP.yaml --scheduler greedy --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" --cluster-config /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/cluster_bCLIP.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'

## PRO-seq

In [14]:
command = """snakemake \
--snakefile """+subdirs['wf_dir']+"""Snakefile_PROseq \
--configfile """+subdirs['wf_dir']+"""config_PROseq.yaml \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" \
--cluster-config """+subdirs['wf_dir']+"""cluster_PROseq.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/Snakefile_PROseq --configfile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/config_PROseq.yaml --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" --cluster-config /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/cluster_PROseq.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'

## RNA-seq

In [30]:
command = """snakemake \
--snakefile """+subdirs['wf_dir']+"""Snakefile_RNAseq \
--configfile """+subdirs['wf_dir']+"""config_RNAseq.yaml \
--printshellcmds \
--use-conda --conda-frontend conda \
--use-singularity \
--singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" \
--cluster-config """+subdirs['wf_dir']+"""cluster_RNAseq.json \
--cores 500 \
--local-cores 10 \
--jobs 100 \
--latency-wait 60 \
--cluster "sbatch \
--cpus-per-task={cluster.threads} \
--mem={cluster.mem} \
--qos={cluster.queue} \
--partition={cluster.partition} \
--time={cluster.time} \
--output={cluster.out}" \
--nolock \
-np"""
command

'snakemake --snakefile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/Snakefile_RNAseq --configfile /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/config_RNAseq.yaml --printshellcmds --use-conda --conda-frontend conda --use-singularity --singularity-args "--bind /scicore/home/zavolan/mirono0000/Projects/bCLIP/,/scicore/home/zavolan/GROUP/StefanieCLIP/,/scicore/home/zavolan/GROUP/Genomes/" --cluster-config /scicore/home/zavolan/mirono0000/Projects/bCLIP/bclip_workflow/cluster_RNAseq.json --cores 500 --local-cores 10 --jobs 100 --latency-wait 60 --cluster "sbatch --cpus-per-task={cluster.threads} --mem={cluster.mem} --qos={cluster.queue} --partition={cluster.partition} --time={cluster.time} --output={cluster.out}" --nolock -np'

## Auxiliary

In [ ]:
# run if want to remove all bam files
sample_file_name = subdirs['temp_dir']+'bam_file_paths.tsv'

os.system("""find """+'/scicore/home/zavolan/GROUP/StefanieCLIP/aleksei/output/'+""" -name '*.Aligned.sortedByCoord.out.bam' > """+sample_file_name)
bam_file_paths = pd.read_csv(sample_file_name,delimiter="\t",
                                   index_col=None,header=None)
bam_file_paths['dir'] = bam_file_paths.apply(lambda x:os.path.dirname(x[0]), 1)
command = 'rm -r '+' '.join(bam_file_paths['dir'])
# out = subprocess.check_output(command, shell=True)